https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='cuda:0')


In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.7333, -0.6480, -0.3576,  0.3679, -0.3662, -0.1141,  0.0413, -0.0099,
          0.6802,  0.1362,  0.2949, -0.7248,  0.2901, -0.5111,  0.2599, -0.2306,
         -0.2197,  0.0712,  0.0035, -0.2118],
        [ 0.1580, -0.4751,  0.0948, -0.0507, -0.1669, -0.1430, -0.0318,  0.1958,
          0.5407,  0.0135,  0.3833, -0.4919,  0.4817, -0.2002,  0.3376, -0.3991,
         -0.0773,  0.0909,  0.3184, -0.1551],
        [-0.3224, -0.6137, -0.1792,  0.1392, -0.2488, -0.4118, -0.0177,  0.2717,
          0.1202,  0.1335,  0.2427, -0.4853,  0.2409, -0.6465,  0.0540,  0.0031,
          0.1079,  0.3079,  0.3942, -0.1364]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.3679, 0.0000, 0.0000, 0.0413, 0.0000, 0.6802,
         0.1362, 0.2949, 0.0000, 0.2901, 0.0000, 0.2599, 0.0000, 0.0000, 0.0712,
         0.0035, 0.0000],
        [0.1580, 0.0000, 0.0948, 0.0000, 0.0000, 0.0000, 0.0000, 0.1958, 0.5407,
         0.0135, 0.3833, 0.0000, 0.4817, 0.0000, 0.33

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0227, -0.0145,  0.0274,  ..., -0.0230, -0.0182,  0.0279],
        [ 0.0060, -0.0282,  0.0326,  ..., -0.0205, -0.0325,  0.0150]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0048,  0.0180], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0385, -0.0220,  0.0426,  ..., -0.0392, -0.0181, -0.0046],
        [ 0.0247,  0.0393,  0.0152,  ..., -0.0337,  0.0048, -0.0338]],
       device='cuda:0', grad_fn=<Sl